In [1]:
print('Initializing data engineering!')
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import json, requests

Initializing data engineering!


### World data engineering
#### Fetching worldwide data

In [2]:
# df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')
# df = pd.read_json('https://covidapi.info/api/v1/country/BRA')
# df = pd.read_json('https://api.covid19api.com/dayone/country/brazil/status/confirmed')
#https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#cc76052f-6601-4645-80e5-ca7aaa36f8ef
df_countries = pd.read_csv('../data/world_countries_2019.csv')

df = pd.DataFrame()
url = "https://pomber.github.io/covid19/timeseries.json"
req = requests.get(url)
# r = r.json()
j = json.loads(req.text)

#### Fetching countries's pandemic data from Pomber's JSON to a dataframe 

In [3]:
# Loading countries names to dict
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Loading countries data do dict then to dataframe
dic = []
for country in countries:
    i = 0
    while i < len(j[country]):
        if j[country][i]['confirmed'] == 0:
            i += 1
            continue
        row = {'country': country, 'date': j[country][i]['date'], 'cases':j[country][i]['confirmed'],
               'deaths':j[country][i]['deaths'], 'recoveries':j[country][i]['recovered']}
        dic.append(row)
        i += 1 
df = pd.DataFrame.from_dict(dic)
df[df['country']=='Brazil'].tail()

,country,date,cases,deaths,recoveries
1312,Brazil,2020-4-26,63100,4286,30152
1313,Brazil,2020-4-27,67446,4603,31142
1314,Brazil,2020-4-28,73235,5083,32544
1315,Brazil,2020-4-29,79685,5513,34132
1316,Brazil,2020-4-30,87187,6006,35935


#### Feature engineering

In [4]:
for country in countries:
    qtdeDays = len(df[df.country == country])+1
    df.loc[df.country == country, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.country == country, 'case_day'] = df[df.country == country]['cases'].diff()    
    df.loc[df.country == country, 'death_day'] = df[df.country == country]['deaths'].diff()
    df.loc[df.country == country, 'recovery_day'] = df[df.country == country]['recoveries'].diff()

    # % daily variations
    df.loc[df.country == country, '%var_case_day'] = ((df[df.country == country]['case_day'] - df[df.country == country]['case_day'].shift()) / df[df.country == country]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_death_day'] = ((df[df.country == country]['death_day'] - df[df.country == country]['death_day'].shift()) / df[df.country == country]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_recovery_day'] = ((df[df.country == country]['recovery_day'] - df[df.country == country]['recovery_day'].shift()) / df[df.country == country]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.country == country) & (df.day == 1), 'case_day']= df.loc[(df.country == country) & (df.day==1), 'cases']
    df.loc[(df.country == country) & (df.day == 1), 'death_day']= df.loc[(df.country == country) & (df.day==1), 'deaths']
    df.loc[(df.country == country) & (df.day == 1), 'recovery_day']= df.loc[(df.country == country) & (df.day==1), 'recoveries']
    
    # Buscando a população do país e calculado os indicador per milhão
    if not df_countries[df_countries['country']==country].empty:
        million = df_countries[df_countries['country']==country]['PopTotal'].values[0] / 1000
        cases_million = (df[df.country == country]['case_day'] / million).round(1)
        deaths_million = (df[df.country == country]['death_day'] / million).round(1)
        recoveries_million = (df[df.country == country]['recovery_day'] / million).round(1)
    else:
        cases_million = 0
        deaths_million = 0
        recoveries_million = 0
        
    df.loc[df.country == country, 'cases_million'] = cases_million
    df.loc[df.country == country, 'deaths_million'] = deaths_million
    df.loc[df.country == country, 'recoveries_million'] = recoveries_million
    
    # moving averages (from last 7 days)
    df.loc[df.country == country, 'avg7_cases'] = df[df.country == country]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths'] = df[df.country == country]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries'] = df[df.country == country]['recovery_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    
    df.loc[df.country == country, 'avg7_cases_million'] = df[df.country == country]['cases_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths_million'] = df[df.country == country]['deaths_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries_million'] = df[df.country == country]['recoveries_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
10750,Yemen,2020-4-28,1,0,1,19,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,100.00,0
10751,Yemen,2020-4-29,6,0,1,20,5,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,16.67,5
10752,Yemen,2020-4-30,6,2,1,21,0,2,0,-100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.33,16.67,3
10753,Comoros,2020-4-30,1,0,0,1,1,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1
10754,Tajikistan,2020-4-30,15,0,0,1,15,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,15


#### Saving CSV

In [5]:
df.to_csv('../data/world_corona19_data.csv', index = False)

In [6]:
df[df['country']=='Brazil'].tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
1312,Brazil,2020-4-26,63100,4286,30152,61,3776,229,992,-28.50,...,4.7,3492.0,260.0,1146.0,16.0,1.0,5.0,6.79,47.78,28662
1313,Brazil,2020-4-27,67446,4603,31142,62,4346,317,990,15.10,...,4.7,3814.0,288.0,1287.0,18.0,1.0,6.0,6.82,46.17,31701
1314,Brazil,2020-4-28,73235,5083,32544,63,5789,480,1402,33.20,...,6.6,4308.0,334.0,1364.0,20.0,1.0,6.0,6.94,44.44,35608
1315,Brazil,2020-4-29,79685,5513,34132,64,6450,430,1588,11.42,...,7.5,4846.0,372.0,1259.0,22.0,1.0,5.0,6.92,42.83,40040
1316,Brazil,2020-4-30,87187,6006,35935,65,7502,493,1803,16.31,...,8.5,5307.0,382.0,1337.0,25.0,1.0,6.0,6.89,41.22,45246


#### countries not located in UN dataset

In [7]:
for country in countries:
    if df_countries[df_countries['country']==country]['PopTotal'].empty:
        print(country)
# df[df['pais'] == pais].tail()
# df_countries[df_countries['Location']==pais]['PopTotal']

Bosnia and Herzegovina
Brunei
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Diamond Princess
Eswatini
Holy See
Iran
Korea, South
Moldova
North Macedonia
Russia
Taiwan*
Tanzania
United Arab Emirates
Venezuela
Vietnam
Syria
Laos
Libya
West Bank and Gaza
Kosovo
Burma
MS Zaandam
Comoros
Tajikistan


### Brazil data engineering

In [8]:
df = pd.read_csv('../data/gov_brazil_corona19_data.csv', sep=';')
df.rename(columns={'regiao': 'region', 'estado':'state', 'data':'date','casosNovos': 'case_day', 'casosAcumulados':'cases', 'obitosNovos':'death_day','obitosAcumulados':'deaths'}, inplace= True)
df['date'] = df['date'].astype('datetime64[ns]')

df.tail()

,region,state,date,case_day,cases,death_day,deaths
2479,Centro-Oeste,DF,2020-04-26,53,1066,1,27
2480,Centro-Oeste,DF,2020-04-27,80,1146,0,27
2481,Centro-Oeste,DF,2020-04-28,67,1213,1,28
2482,Centro-Oeste,DF,2020-04-29,62,1275,0,28
2483,Centro-Oeste,DF,2020-04-30,81,1356,2,30


#### Feature engineering

In [9]:
states = df.state.unique()
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for state in states:
    qtdeDays = len(df[df.state == state])+1
    df.loc[df.state == state, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.state == state, 'case_day'] = df[df.state == state]['cases'].diff()    
    df.loc[df.state == state, 'death_day'] = df[df.state == state]['deaths'].diff()
#     df.loc[df.state == state, 'recovery_day'] = df[df.state == state]['recoveries'].diff()

    # % daily variations
    df.loc[df.state == state, '%var_case_day'] = ((df[df.state == state]['case_day'] - df[df.state == state]['case_day'].shift()) / df[df.state == state]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.state == state, '%var_death_day'] = ((df[df.state == state]['death_day'] - df[df.state == state]['death_day'].shift()) / df[df.state == state]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
#     df.loc[df.state == state, '%var_recovery_day'] = ((df[df.state == state]['recovery_day'] - df[df.state == state]['recovery_day'].shift()) / df[df.state == state]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.state == state) & (df.day == 1), 'case_day']= df.loc[(df.state == state) & (df.day==1), 'cases']
    df.loc[(df.state == state) & (df.day == 1), 'death_day']= df.loc[(df.state == state) & (df.day==1), 'deaths']
    
    # moving averages (from last 7 days)
    df.loc[df.state == state, 'avg7_cases'] = df[df.state == state]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'avg7_deaths'] = df[df.state == state]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'perc_death'] = (df[df.state == state]['deaths']/df[df.state == state]['cases']*100).round(2) 
    df.loc[df.state == state, 'avg7_perc_death'] = df[df.state == state]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
# df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
# df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
# df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,region,state,date,case_day,cases,death_day,deaths,day,%var_case_day,%var_death_day,avg7_cases,avg7_deaths,perc_death,avg7_perc_death
2479,Centro-Oeste,DF,2020-04-26,53,1066,1,27,51,120.83,0.0,34.0,0.0,2.53,2.63
2480,Centro-Oeste,DF,2020-04-27,80,1146,0,27,52,50.94,-100.0,39.0,0.0,2.36,2.58
2481,Centro-Oeste,DF,2020-04-28,67,1213,1,28,53,-16.25,0.0,47.0,0.0,2.31,2.52
2482,Centro-Oeste,DF,2020-04-29,62,1275,0,28,54,-7.46,-100.0,47.0,0.0,2.20,2.46
2483,Centro-Oeste,DF,2020-04-30,81,1356,2,30,55,30.65,0.0,56.0,0.0,2.21,2.40


In [10]:
df.to_csv('../data/brazil_corona19_data.csv', index = False)

### São Paulo data engineering

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
df.rename(columns={'munic': 'city', 'casos': 'cases','obitos':'deaths'}, inplace= True)
df['date'] = pd.to_datetime(dict(year=2020, month=df.mes, day=df.dia))
df.fillna(0, inplace = True)
df.drop(['dia','mes'], inplace=True, axis=1)

df = df[~df['city'].isin(['ignorado','outro estado', 'total geral', 'outro pais'])] 

df.tail()

,city,cases,deaths,codigo_ibge,latitude,longitude,date
5707,varzea paulista,13.0,2.0,3556503.0,-23.2136,-46.8234,2020-04-29
5708,vinhedo,29.0,2.0,3556701.0,-23.0302,-46.9833,2020-04-29
5709,vista alegre do alto,1.0,0.0,3556909.0,-21.1692,-48.6284,2020-04-29
5710,votorantim,12.0,3.0,3557006.0,-23.5446,-47.4388,2020-04-29
5711,votuporanga,18.0,0.0,3557105.0,-20.4237,-49.9781,2020-04-29


#### Feature engineering

In [12]:
cities = df.city.unique()
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for city in cities:
    qtdeDays = len(df[df.city == city])+1
    df.loc[df.city == city, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.city == city, 'case_day'] = df[df.city == city]['cases'].diff()    
    df.loc[df.city == city, 'death_day'] = df[df.city == city]['deaths'].diff()
#     df.loc[df.city == city, 'recovery_day'] = df[df.city == city]['recoveries'].diff()

    # % daily variations
    df.loc[df.city == city, '%var_case_day'] = ((df[df.city == city]['case_day'] - df[df.city == city]['case_day'].shift()) / df[df.city == city]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.city == city, '%var_death_day'] = ((df[df.city == city]['death_day'] - df[df.city == city]['death_day'].shift()) / df[df.city == city]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
#     df.loc[df.city == city, '%var_recovery_day'] = ((df[df.city == city]['recovery_day'] - df[df.city == city]['recovery_day'].shift()) / df[df.city == city]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.city == city) & (df.day == 1), 'case_day']= df.loc[(df.city == city) & (df.day==1), 'cases']
    df.loc[(df.city == city) & (df.day == 1), 'death_day']= df.loc[(df.city == city) & (df.day==1), 'deaths']
    
    # moving averages (from last 7 days)
    df.loc[df.city == city, 'avg7_cases'] = df[df.city == city]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.city == city, 'avg7_deaths'] = df[df.city == city]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.city == city, 'perc_death'] = (df[df.city == city]['deaths']/df[df.city == city]['cases']*100).round(2) 
    df.loc[df.city == city, 'avg7_perc_death'] = df[df.city == city]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
# df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
# df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
# df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,city,cases,deaths,codigo_ibge,latitude,longitude,date,day,case_day,death_day,%var_case_day,%var_death_day,avg7_cases,avg7_deaths,perc_death,avg7_perc_death
5707,varzea paulista,13.0,2.0,3556503.0,-23.2136,-46.8234,2020-04-29,16,0,0,-100.0,0.0,1.0,0.0,15.38,20.35
5708,vinhedo,29.0,2.0,3556701.0,-23.0302,-46.9833,2020-04-29,28,2,0,0.0,0.0,2.0,0.0,6.90,6.05
5709,vista alegre do alto,1.0,0.0,3556909.0,-21.1692,-48.6284,2020-04-29,9,0,0,0.0,0.0,0.0,0.0,0.00,0.00
5710,votorantim,12.0,3.0,3557006.0,-23.5446,-47.4388,2020-04-29,28,0,0,-100.0,-100.0,0.0,0.0,25.00,24.40
5711,votuporanga,18.0,0.0,3557105.0,-20.4237,-49.9781,2020-04-29,28,3,0,-400.0,0.0,1.0,0.0,0.00,0.00


In [13]:
df.to_csv('../data/saoPaulo_corona19_data.csv', index = False)

In [14]:
# df[df['country']=='Belgium']
print('Data engineering done!')

Data engineering done!
